# War Modeling
## Money or Soldiers : What's more important

<img src="https://ipython-books.github.io/pages/chapter06_viz/04_d3_files/D3.png" width=500px>

**Group members:** 
<br>Jonathan Le Roux
<br>Pranav Agarwal
<br>Bhavya Jain
<br>Ethan Hecht

In [ ]:
class Territory:
    """
    Class that houses the information for each individual territory of a country
    
    Attributes:
    position - a list in the form longitude, latitude
    num_soldiers - a numeric value which has the number of soldiers a territory has in thousands
    
    
    Methods:
    add_soldiers - increases the number of soldiers and adds the new soldiers skill levels to the dictionary
    attack = fights oppositing territory given in the argument, updates the teritories army
             after war and returns the opposing territory with an updated army. 
    """
    def __init__(self,name, position, num_soldiers):
        import numpy as np
        
        self.position = position
        self.name = name
        self.money = 0
        #below sets each soldier skill level randomly
        self.skill_levels =  list(np.random.normal(0.5, 0.1, num_soldiers))
               
    def get_position(self):
        return self.position
    
    def set_position(self, position):
        self.position = position
    
    def get_num_soldiers(self):
        return len(self.skill_levels)
   
    def add_money(amount):
        self.money += amount
    
    def remove_money(amount):
        self.money-= amount
    
    def add_soldiers(self, number_adding):
        self.num_soldiers += number_adding
        for i in range(self.num_soldiers-number_adding+1,self.num_soldiers+1):
             self.skill_levels = self.skill_levels.append(random.uniform(0,1))
            
    def attack(self,opposing_territory):
        import numpy as np
        #returns 2 territory objects
        army1 = np.copy(self.skill_levels)
        army2 = np.copy(opposing_territory.skill_levels)
        #randomizing the attack pattern of two armies
        np.random.shuffle(army1) 
        np.random.shuffle(army2)
        death1, death2 = 0,0
       
        #run loop for minimum length steps till one of the army dies completely
        while army1.size!=0 and army2.size!=0:
            soldier1, soldier2 = army1[0], army2[0]
            probability = np.random.random()
            if soldier1/(soldier1 + soldier2) > probability:
                army2 = np.delete(army2, 0)
                death2 += 1
            else:
                army1 = np.delete(army1, 0)
                death1 += 1
        self.skill_levels = army1
        opposing_territory.skill_levels = army2
        return opposing_territory 
    
    def to_string(self):
        return ("Name: " +  str(self.name)+  " is at position"+  str(self.position) + "with " + str(self.get_num_soldiers())+  " soldiers")
        

In [ ]:
class Country:
    
    def __init__(self, territory_list, money):
        import numpy as np
        self.territory_list = territory_list
    # Territory list will be initialzied as an attribute (List will come from terriotry class that Jonathan will make)
        self.money = money
    # Will be a number of money for the entire country in millions (again initialized once composition is used with Jonathan's territory class)
        self.money_array = np.array([self.money/len(self.territory_list)]*len(self.territory_list))
    
    def add_territory(self,territory, money):
        import numpy as np
        self.territory_list.append(territory)
        self.money_array = np.append(self.money_array,money)
        self.dist_money()
        
    def remove_territory(self,territory): 
        new_list_of_territories = []
        new_list_of_money = []
        for i in range(len(self.territory_list)):
            if self.territory_list[i].name != territory.name:
                new_list_of_territories.append(self.territory_list[i])
                new_list_of_money.append(self.money_array[i])
        self.territory_list = new_list_of_territories
        self.money_array = new_list_of_money
    
    def dist_money(self):
        import numpy as np
        # Once a territory wins a battle the country needs to redistribute the money it has evenly over the numbe of territories
        money_per_territory = self.money / len(self.territory_list)
        self.money_array = np.array([money_per_territory]* len(self.territory_list))
        
    
    def add_GDP(self, num_iterations):
        if num_iterations % 10 ==0:
            # Every 10 iterations of the simulation GDP will be added to each country
            self.money_array = 1.2 * self.money_array          
    
    def lose_money_winner(self,winning_territory):
        # When a territory participates in a battle they will lose money whether they win or lose
        for i in range(len(self.territory_list)):
            if self.territory_list[i].name == winning_territory.name:
                self.money_array[i] = self.money_array[i]*0.9
      
    def lose_money_loser(self,losing_territory):
        # When a territory loses a battle after participating they will lose all the monet they have
        for i in range(len(self.territory_list)):
            if self.territory_list[i].name == losing_territory.name:
                self.money_array[i] = 0
                
    def to_string(self):
        temp = ""
        for i in self.territory_list:
            temp += i.to_string()  + "\n"
            
        return temp
    

In [ ]:
#Exploration code to show how to do individual battles
#uncomment as necessary

#A = Territory("A",[1,0],1000)
#B = Territory("B",[0,1],9000)
#C = Territory("C",[2,0],5000)
#D = Territory("D",[0,2],1500)
#E = Territory("E",[3,0],10000)
#F = Territory("F",[0,3],4500)

#china = Country([A,B,C],120000)
#russia = Country([D,E,F],100000)


#russia.territory_list[0].skill_levels = china.territory_list[0].attack(russia.territory_list[0]).skill_levels
#len(russia.territory_list[0].skill_levels)

#russia.lose_money_winner(russia.territory_list[0])
#china.lose_money_loser(china.territory_list[0])

#russia.add_territory(china.territory_list[0],china.money_array[0])
#china.remove_territory(china.territory_list[0])

#russia.money_array
#china.money_array

In [ ]:
"""
This is going to be our data for the simulation at least initially
"""

import pandas as pd
import numpy as np

territory_data = pd.read_csv("MOCK_DATA (1).csv")
territory_data = territory_data.replace(0,"Russia")
territory_data = territory_data.replace(1,"China")

country_1_territory_list = []
country_2_territory_list = []

for i in range(len(territory_data)):
    if territory_data["country"][i] == "Russia":
        temp_Territory = Territory(territory_data["name"][i], [territory_data["longitude"][i],territory_data["latitude"][i] ],territory_data["soldiers"][i])
        country_1_territory_list.append(temp_Territory)
    else:
        temp_Territory = Territory(territory_data["name"][i], [territory_data["longitude"][i],territory_data["latitude"][i] ],territory_data["soldiers"][i])
        country_2_territory_list.append(temp_Territory)
        
Russia = Country(country_1_territory_list,100000)
China = Country(country_2_territory_list,120000)

print("Russian territories: ")
print(Russia.to_string())
print("\nChina territories: ")
print(China.to_string())